# Part 2.2: Vanishing Gradients in RNNs

This notebook demonstrates the vanishing gradient problem in RNNs through experimental analysis.

## Objective
- Use long-sequence memory task (copy task)
- Train vanilla RNN with sequence lengths 20, 50, and 100
- Track gradient magnitudes at different time steps
- Analyze spectral radius of Whh and gradient decay

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader, Dataset

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(42)
np.random.seed(42)

print(f"Using device: {device}")
print("Vanishing Gradient Analysis in RNNs")
print("=" * 40)

In [ ]:
# Copy Task Dataset
class CopyTaskDataset(Dataset):
    def __init__(self, sequence_length, num_samples=1000, vocab_size=10):
        self.sequence_length = sequence_length
        self.num_samples = num_samples
        self.vocab_size = vocab_size
        
        # Generate data
        self.data = self._generate_data()
        
    def _generate_data(self):
        data = []
        for _ in range(self.num_samples):
            # Random sequence
            seq = torch.randint(1, self.vocab_size, (self.sequence_length,))
            # Input: [sequence, zeros, delimiter]
            delimiter = torch.tensor([self.vocab_size])  # Special delimiter token
            zeros = torch.zeros(self.sequence_length, dtype=torch.long)
            
            input_seq = torch.cat([seq, zeros, delimiter])
            target_seq = torch.cat([zeros, seq, torch.tensor([0])])
            
            data.append((input_seq, target_seq))
        
        return data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

# Vanilla RNN with gradient tracking
class VanillaRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=1):
        super(VanillaRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.embedding = nn.Embedding(vocab_size + 1, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, num_layers, 
                          batch_first=True, nonlinearity='tanh')
        self.fc = nn.Linear(hidden_size, vocab_size + 1)
        
        # Store gradients for analysis
        self.gradient_norms = []
        
    def forward(self, x, hidden=None):
        embedded = self.embedding(x)
        output, hidden = self.rnn(embedded, hidden)
        output = self.fc(output)
        return output, hidden
    
    def get_spectral_radius(self):
        """Calculate spectral radius of hidden-to-hidden weight matrix"""
        Whh = self.rnn.weight_hh_l0.data.cpu().numpy()
        eigenvalues = np.linalg.eigvals(Whh)
        return np.max(np.abs(eigenvalues))

print("Copy task and RNN classes defined")

In [ ]:
# Training function with gradient tracking
def train_and_track_gradients(model, dataloader, num_epochs=100):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    losses = []
    gradient_norms_per_timestep = []
    
    model.train()
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        epoch_gradients = []
        
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            outputs, _ = model(inputs)
            
            # Compute loss
            loss = criterion(outputs.view(-1, outputs.size(-1)), targets.view(-1))
            
            # Backward pass
            loss.backward(retain_graph=True)
            
            # Track gradient norms at different timesteps
            if epoch % 20 == 0:  # Sample every 20 epochs
                timestep_grads = []
                
                # Approximate gradient flow by looking at parameter gradients
                for name, param in model.named_parameters():
                    if 'weight_hh' in name and param.grad is not None:
                        grad_norm = param.grad.norm().item()
                        timestep_grads.append(grad_norm)
                
                epoch_gradients.append(timestep_grads)
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
            
            optimizer.step()
            epoch_loss += loss.item()
        
        losses.append(epoch_loss / len(dataloader))
        
        if epoch_gradients:
            gradient_norms_per_timestep.append(epoch_gradients)
        
        if (epoch + 1) % 25 == 0:
            print(f'Epoch {epoch+1}: Loss = {losses[-1]:.4f}')
    
    return losses, gradient_norms_per_timestep

# Experiment with different sequence lengths
sequence_lengths = [20, 50, 100]
results = {}

print("\nStarting experiments with different sequence lengths...")

for seq_len in sequence_lengths:
    print(f"\n🔄 Training on sequence length: {seq_len}")
    print("-" * 40)
    
    # Create dataset and dataloader
    dataset = CopyTaskDataset(sequence_length=seq_len, num_samples=500)
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Create model
    model = VanillaRNN(vocab_size=10, hidden_size=64).to(device)
    spectral_radius = model.get_spectral_radius()
    
    print(f"Initial spectral radius: {spectral_radius:.3f}")
    
    # Train and track
    losses, gradients = train_and_track_gradients(model, dataloader, num_epochs=100)
    
    # Final spectral radius
    final_spectral_radius = model.get_spectral_radius()
    
    results[seq_len] = {
        'losses': losses,
        'gradients': gradients,
        'initial_spectral_radius': spectral_radius,
        'final_spectral_radius': final_spectral_radius,
        'model': model
    }
    
    print(f"✅ Completed. Final spectral radius: {final_spectral_radius:.3f}")

print("\n🎉 All experiments completed!")

In [ ]:
# Comprehensive visualization and analysis
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Vanishing Gradient Analysis in RNNs', fontsize=16, fontweight='bold')

colors = ['blue', 'orange', 'green']

# 1. Training Loss Curves
ax = axes[0, 0]
for i, seq_len in enumerate(sequence_lengths):
    losses = results[seq_len]['losses']
    ax.plot(losses, color=colors[i], linewidth=2, label=f'Length {seq_len}')

ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('Training Loss vs Sequence Length')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_yscale('log')

# 2. Spectral Radius Analysis
ax = axes[0, 1]
initial_sr = [results[sl]['initial_spectral_radius'] for sl in sequence_lengths]
final_sr = [results[sl]['final_spectral_radius'] for sl in sequence_lengths]

x = np.arange(len(sequence_lengths))
width = 0.35

ax.bar(x - width/2, initial_sr, width, label='Initial', alpha=0.8)
ax.bar(x + width/2, final_sr, width, label='Final', alpha=0.8)
ax.axhline(y=1.0, color='red', linestyle='--', alpha=0.7, label='Stability Threshold')

ax.set_xlabel('Sequence Length')
ax.set_ylabel('Spectral Radius')
ax.set_title('Spectral Radius of Whh Matrix')
ax.set_xticks(x)
ax.set_xticklabels(sequence_lengths)
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Eigenvalue Visualization for longest sequence
ax = axes[0, 2]
model_100 = results[100]['model']
Whh = model_100.rnn.weight_hh_l0.data.cpu().numpy()
eigenvals = np.linalg.eigvals(Whh)

ax.scatter(eigenvals.real, eigenvals.imag, alpha=0.7, s=60)
circle = plt.Circle((0, 0), 1, fill=False, color='red', linestyle='--', linewidth=2)
ax.add_patch(circle)

ax.set_xlabel('Real Part')
ax.set_ylabel('Imaginary Part')
ax.set_title('Eigenvalues of Whh (Length 100)')
ax.grid(True, alpha=0.3)
ax.set_aspect('equal')

# 4. Learning Performance vs Sequence Length
ax = axes[1, 0]
final_losses = [results[sl]['losses'][-1] for sl in sequence_lengths]

bars = ax.bar(sequence_lengths, final_losses, color=colors, alpha=0.7)
ax.set_xlabel('Sequence Length')
ax.set_ylabel('Final Loss')
ax.set_title('Learning Difficulty vs Sequence Length')
ax.grid(True, alpha=0.3)
ax.set_yscale('log')

# Add value labels
for bar, loss in zip(bars, final_losses):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height * 1.1,
           f'{loss:.3f}', ha='center', va='bottom', fontweight='bold')

# 5. Gradient Flow Analysis
ax = axes[1, 1]

# Simulate gradient decay
time_steps = np.arange(1, 101)
for i, seq_len in enumerate(sequence_lengths):
    sr = results[seq_len]['final_spectral_radius']
    # Approximate gradient magnitude decay
    grad_magnitude = np.power(sr, time_steps)
    ax.plot(time_steps, grad_magnitude, color=colors[i], 
           linewidth=2, label=f'Length {seq_len} (ρ={sr:.2f})')

ax.set_xlabel('Time Steps Back')
ax.set_ylabel('Relative Gradient Magnitude')
ax.set_title('Gradient Decay Over Time')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_yscale('log')

# 6. Summary Table
ax = axes[1, 2]
ax.axis('tight')
ax.axis('off')

table_data = []
for seq_len in sequence_lengths:
    result = results[seq_len]
    table_data.append([
        f'{seq_len}',
        f"{result['final_spectral_radius']:.3f}",
        f"{result['losses'][-1]:.3f}",
        'Yes' if result['losses'][-1] > 1.0 else 'No'
    ])

table = ax.table(cellText=table_data,
                colLabels=['Seq Length', 'Spectral Radius', 'Final Loss', 'Vanishing?'],
                cellLoc='center',
                loc='center')
table.auto_set_font_size(False)
table.set_fontsize(12)
table.scale(1.2, 2)
ax.set_title('Vanishing Gradient Summary', fontweight='bold')

plt.tight_layout()
plt.show()

## Analysis: Vanishing Gradient Problem in RNNs

### Key Findings

1. **Sequence Length Impact**: Longer sequences show degraded learning performance
2. **Spectral Radius**: Values close to or above 1.0 indicate potential instability
3. **Gradient Decay**: Exponential decay of gradients over long sequences
4. **Learning Failure**: Copy task becomes impossible for very long sequences

### Mathematical Foundation

The vanishing gradient problem stems from the recursive nature of RNN computation:
- Gradients flow backward through matrix multiplications
- Each step involves multiplication by the weight matrix Whh
- If spectral radius < 1, gradients vanish exponentially
- If spectral radius > 1, gradients explode exponentially

### Solutions

- **LSTM/GRU**: Gating mechanisms create gradient highways
- **Gradient Clipping**: Prevents exploding gradients
- **Careful Initialization**: Initialize weights to have spectral radius ≈ 1
- **Skip Connections**: Provide direct gradient paths